In [1]:
import pickle
import datetime
import os
import requests
import scipy
import math
import time
import multiprocessing
import itertools
import json
import warnings
import glob

import numpy as np
import pandas as pd

import sys
from data_utils import *
from feature import *

In [2]:
## 填你的账号密码
user = ''
password = ''

In [3]:
trading_time_interval_dict = {
    'night': (3600 * -3, 3600 * -1),
    'day1': (3600 * 9, 3600 * 10 + 60 * 15),
    'day2': (3600 * 10 + 60 * 30, 3600 * 11 + 60 * 30),
    'day3': (3600 * 13 + 60 * 30, 3600 * 15)
}
volume_multiple = 10
tick_size_in_yuan = 1
symbol_class = 'rb'
exchange = 'SHFE'

# 加载原始数据

In [4]:
manager = multiprocessing.Manager()
shared_raw_data_dict = manager.dict()

start_time = datetime.datetime.today() - datetime.timedelta(40)
end_date = (datetime.datetime.today() - datetime.timedelta(20)).strftime('%Y-%m-%d')

In [5]:
%%time
is_new = False

process_list = list()
i = 0
while True:
    date = (start_time + datetime.timedelta(i)).strftime('%Y-%m-%d')
    i += 1
    if date > end_date:
        break
    #max_symbol = get_max_symbol_new(date, symbol_class, exchange, user, password)
    max_symbol = get_max_symbol(date, symbol_class)
    print(date, max_symbol)
    
    process = multiprocessing.Process(target=append_data, args=(shared_raw_data_dict, date, max_symbol, trading_time_interval_dict,
                                                                is_new, exchange, user, password, volume_multiple))
    process.start()
    process_list.append(process)
    
for process in process_list:
    process.join()

2023-01-25 
2023-01-26 
2023-01-27 
2023-01-28 
2023-01-29 
2023-01-30 rb2305
2023-01-31 rb2305
2023-02-01 rb2305
2023-02-02 rb2305
2023-02-03 rb2305
2023-02-04 
2023-02-05 
2023-02-06 rb2305
2023-02-07 rb2305
2023-02-08 rb2305
2023-02-09 rb2305
2023-02-10 rb2305
2023-02-11 
2023-02-12 
2023-02-13 rb2305
2023-02-14 rb2305
Wall time: 7.41 s


In [37]:
lag1_col_list = []
for i in range(1, 6):
    lag1_col_list.append('BidPrice%d' % i)
    lag1_col_list.append('AskPrice%d' % i)
    lag1_col_list.append('BidVolume%d' % i)
    lag1_col_list.append('AskVolume%d' % i)
    
diff_len1_col_list = ['Turnover', 'Volume']

In [38]:
def process_feature(df):
    for col in lag1_col_list:
        df['%s_Lag1' % col] = df[col].shift(1)
    for col in diff_len1_col_list:
        df['%s_DiffLen1' % col] = df[col].diff(1)
    for level in range(1, 6):   
        df['MidPriceLvl%d' % level] = df.apply(lambda x: cal_mid_price(x, level), axis=1)
        df['MicroPriceLvl%d' % level] = df.apply(lambda x: cal_micro_price(x, level), axis=1)
    df['VWAP'] = df.apply(lambda x: cal_vwap(x), axis=1)
    df['MarketBuyVolume'] = df.apply(lambda x: cal_market_buy_volume(x), axis=1)
    df['MarketSellVolume'] = df.apply(lambda x: cal_market_sell_volume(x), axis=1)
    
    for accum_col in ['MarketBuyVolume', 'MarketSellVolume']:
        append_accum_feature(df, accum_col)
        
    for length in [1, 20, 40, 60]:
        df['ReturnLen%d' % length] = df['MidPriceLvl1'].shift(-length) - df['MidPriceLvl1']
    
    return df


def append_feature(shared_feature_data_dict, shared_data_dict, key):
    daily_dict = dict()
    for session in SESSION_NAME_LIST:
        session_df = shared_data_dict[key][session]
        if len(session_df) > 0:
            session_df = process_feature(session_df)
        daily_dict[session] = session_df
    shared_feature_data_dict[key] = daily_dict

In [20]:
shared_feature_data_dict = manager.dict()

In [26]:
ls1=list(shared_raw_data_dict.values())

In [31]:
shared_data_dict=shared_raw_data_dict

In [30]:
key=list(shared_raw_data_dict.keys())[0]

In [32]:
shared_feature_data_dict=dict()

In [39]:
daily_dict = dict()
for session in SESSION_NAME_LIST:
    session_df = shared_data_dict[key][session]
    if len(session_df) > 0:
        session_df = process_feature(session_df)
    daily_dict[session] = session_df
shared_feature_data_dict[key] = daily_dict

In [42]:
list(shared_feature_data_dict.values())[0]

{'night': Empty DataFrame
 Columns: [TradingDay, Symbol, ExchTimeOffsetUs, Volume, Turnover, OpenInterest, BidPrice1, BidVolume1, BidCount1, BidPrice2, BidVolume2, BidCount2, BidPrice3, BidVolume3, BidCount3, BidPrice4, BidVolume4, BidCount4, BidPrice5, BidVolume5, BidCount5, AskPrice1, AskVolume1, AskCount1, AskPrice2, AskVolume2, AskCount2, AskPrice3, AskVolume3, AskCount3, AskPrice4, AskVolume4, AskCount4, AskPrice5, AskVolume5, AskCount5, LocalTimeStamp, LocalTime, VolumeMultiple, Session]
 Index: []
 
 [0 rows x 40 columns],
 'day1':       TradingDay  Symbol  ExchTimeOffsetUs  Volume     Turnover  OpenInterest  \
 0     2023-01-30  rb2305           32400.5   14543    615221590       1812163   
 1     2023-01-30  rb2305           32401.0   18955    802247980       1811715   
 2     2023-01-30  rb2305           32401.5   22967    972585400       1812880   
 3     2023-01-30  rb2305           32402.0   24856   1052638930       1813021   
 4     2023-01-30  rb2305           32402.5   

In [33]:
def append_feature(shared_feature_data_dict, shared_data_dict, key):
    daily_dict = dict()
    for session in SESSION_NAME_LIST:
        session_df = shared_data_dict[key][session]
        if len(session_df) > 0:
            session_df = process_feature(session_df)
        daily_dict[session] = session_df
    shared_feature_data_dict[key] = daily_dict

In [34]:
shared_feature_data_dict

{}

In [21]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    process_list = list()
    for key in shared_raw_data_dict.keys():
        print(key)
        process = multiprocessing.Process(target=append_feature, args=(shared_feature_data_dict, shared_raw_data_dict, key))
        process.start()
        process_list.append(process)

    for process in process_list:
        process.join()

('2023-01-30', 'rb2305')
('2023-01-31', 'rb2305')
('2023-02-03', 'rb2305')
('2023-02-01', 'rb2305')
('2023-02-02', 'rb2305')
('2023-02-07', 'rb2305')
('2023-02-08', 'rb2305')
('2023-02-14', 'rb2305')
('2023-02-06', 'rb2305')
('2023-02-09', 'rb2305')
('2023-02-13', 'rb2305')
('2023-02-10', 'rb2305')
Wall time: 197 ms


In [22]:
shared_feature_data_dict.keys()

[]

In [14]:
daily_df_list = list()
for key in shared_feature_data_dict:
    for session_name in SESSION_NAME_LIST:
        daily_df_list.append(shared_feature_data_dict[key][session_name])
all_data = pd.concat(daily_df_list, ignore_index=True)

ValueError: No objects to concatenate

In [11]:
all_data[['TradingDay', FLOAT_TIME_COL, 'Volume', 'MarketBuyVolume', 'MarketSellVolume', 'BidPrice1', 'BidVolume1', 'AskPrice1', 'AskVolume1']]

,TradingDay,ExchTimeOffsetUs,Volume,MarketBuyVolume,MarketSellVolume,BidPrice1,BidVolume1,AskPrice1,AskVolume1
0,2023-01-30,32400.5,14543,NaN,NaN,4230,669,4232,1
1,2023-01-30,32401.0,18955,4412.000000,0.000000,4245,242,4248,15
2,2023-01-30,32401.5,22967,934.000000,3078.000000,4234,2,4236,1
3,2023-01-30,32402.0,24856,1889.000000,0.000000,4239,6,4242,9
4,2023-01-30,32402.5,26557,1701.000000,0.000000,4249,3,4250,4
...,...,...,...,...,...,...,...,...,...
2006175,2023-02-16,53998.0,1877091,36.000000,224.000000,4143,265,4146,149
2006176,2023-02-16,53998.5,1877136,22.333333,22.666667,4143,281,4144,1
2006177,2023-02-16,53999.0,1877144,8.000000,0.000000,4145,9,4146,201
2006178,2023-02-16,53999.5,1877215,33.000000,38.000000,4145,101,4146,178


In [12]:
for col in all_data.columns:
    print(col)

TradingDay
Symbol
ExchTimeOffsetUs
Volume
Turnover
OpenInterest
BidPrice1
BidVolume1
BidCount1
BidPrice2
BidVolume2
BidCount2
BidPrice3
BidVolume3
BidCount3
BidPrice4
BidVolume4
BidCount4
BidPrice5
BidVolume5
BidCount5
AskPrice1
AskVolume1
AskCount1
AskPrice2
AskVolume2
AskCount2
AskPrice3
AskVolume3
AskCount3
AskPrice4
AskVolume4
AskCount4
AskPrice5
AskVolume5
AskCount5
LocalTimeStamp
LocalTime
VolumeMultiple
Session
BidPrice1_Lag1
AskPrice1_Lag1
BidVolume1_Lag1
AskVolume1_Lag1
BidPrice2_Lag1
AskPrice2_Lag1
BidVolume2_Lag1
AskVolume2_Lag1
BidPrice3_Lag1
AskPrice3_Lag1
BidVolume3_Lag1
AskVolume3_Lag1
BidPrice4_Lag1
AskPrice4_Lag1
BidVolume4_Lag1
AskVolume4_Lag1
BidPrice5_Lag1
AskPrice5_Lag1
BidVolume5_Lag1
AskVolume5_Lag1
Turnover_DiffLen1
Volume_DiffLen1
MidPriceLvl1
MicroPriceLvl1
MidPriceLvl2
MicroPriceLvl2
MidPriceLvl3
MicroPriceLvl3
MidPriceLvl4
MicroPriceLvl4
MidPriceLvl5
MicroPriceLvl5
VWAP
MarketBuyVolume
MarketSellVolume
Accum_MarketBuyVolume
Accum_MarketSellVolume
ReturnLen1
